## IMPORTS

In [2]:
from facebook_page_scraper import Facebook_scraper
from facebook_page_scraper import driver_initialization
from selenium.webdriver.common.by import By
from urllib.parse import urljoin
import time
from dotenv import load_dotenv
import os
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.chrome.service import Service
import shutil
import json
import pandas as pd
from bs4 import BeautifulSoup
import csv
from datetime import datetime, timedelta
import re
import random

## WEB CONFIGS

In [3]:
# Specify the correct Geckodriver path
GECKODRIVER_PATH = "/usr/local/bin/geckodriver"  # Adjust as needed

# Configure Firefox options
options = webdriver.FirefoxOptions()
options.binary_location = "/Applications/Firefox.app/Contents/MacOS/firefox"
# options.add_argument("--headless")  # Optional: Remove this line if you want to see the browser

# Start Selenium WebDriver manually
service = Service(GECKODRIVER_PATH)
driver = webdriver.Firefox(service=service, options=options)

WebDriverException: Message: 'geckodriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


## LOGINS

In [ ]:
# Load .env file
load_dotenv("logins.env")

# Facebook credentials from environment variables
fb_email = os.getenv('FACEBOOK_EMAIL')
fb_password = os.getenv('FACEBOOK_PASSWORD')

# Quick check
print(f"Email loaded: {fb_email is not None}, Password loaded: {fb_password is not None}")

## SCRAPER SETTINGS

In [ ]:
# Facebook scraping parameters
page_or_group_name = 409901703293362
#page_group_or_name = 1996906587229548
posts_count = 30
browser = "firefox"
proxy = None
timeout = 300
headless = False  # Should match WebDriver options, please do this so you can log in
isGroup = True

## RUN SCRAPER

In [ ]:
scraper = Facebook_scraper(
    page_or_group_name,
    posts_count,
    browser,
    proxy=proxy,
    timeout=timeout,
    headless=headless,
    isGroup=isGroup,
    username=fb_email,
    password=fb_password
)

## IMPORTANT INSTRUCTIONS

You will NEED to refesh the firefox page that is opened and sign into your account to scrape a private group. 

In [ ]:
json_data = scraper.scrap_to_json()

In [ ]:
print(json_data)

 
 
 
 
 
  

 
 
 
 
 
# PJ'S AWESOME SCRAPER

i got pissed because timestamps weren't working.

the initial parts are what i did to start on the chrome scraping but it got me a little mad when it didn't actually get like half the required posts

### setup: chrome driver, cookies, and facebook group page
- loads the local chromedriver and launches chrome
- injects saved facebook cookies to skip login
- navigates to the target facebook group page

In [67]:
# set paths
CHROMEDRIVER_PATH = "/Users/pjsmaza/Downloads/chromedriver-mac-x64/chromedriver"
COOKIES_PATH = "fb_cookies.json"

# patch facebook_page_scraper to use our local chromedriver
def patched_set_driver_for_browser(self, browser_name):
    options = webdriver.ChromeOptions()
    options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
    # options.add_argument("--headless")  # optional

    service = Service(CHROMEDRIVER_PATH)
    return webdriver.Chrome(service=service, options=options)

# apply the patch
driver_initialization.Initializer.set_driver_for_browser = patched_set_driver_for_browser

 - code below opens a webpage

In [68]:
# set up chrome driver
options = webdriver.ChromeOptions()
options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"

service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=options)

# load facebook and apply session cookies
driver.get("https://www.facebook.com/")
time.sleep(3)  # let it load

with open(COOKIES_PATH, "r") as f:
    cookies = json.load(f)
    for cookie in cookies:
        if "sameSite" in cookie:
            cookie["sameSite"] = "Strict"
        try:
            driver.add_cookie(cookie)
        except Exception as e:
            print("cookie error:", cookie.get("name"), e)

# go to the target group page
driver.get("https://www.facebook.com/groups/1996906587229548/")
time.sleep(5)

### actual scraper function

#### functions: scroll, scrape posts, and save to csv
- scrolls the group page to load posts
- extracts author, timestamp, content, reactions, and post url
- saves everything to a csv file


In [59]:
# pj's ultimate facebook group scraper v2.2
# features:
# - author extraction with 4 fallback methods 
# - relative timestamp conversion (3d → actual date)  
# - reaction scraping with 3 different approaches 
# - error resilience against facebook's nonsense 
# - clean CSV output 


# --------------------------
# 1. SCROLLING MECHANISM
# --------------------------
def scroll_to_load(driver, max_scrolls=600, wait=2, min_scrolls=60, verbose=True):
    """scrolls the page with smart detection but ensures a minimum number of scrolls"""
    last_height = driver.execute_script("return document.body.scrollHeight")
    stall_counter = 0

    for i in range(max_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(wait)

        new_height = driver.execute_script("return document.body.scrollHeight")

        # Count visible posts (optional)
        posts_found = len(driver.find_elements(By.CSS_SELECTOR, "div[role='article']"))
        if verbose:
            print(f"Scroll {i+1}/{max_scrolls} | Posts found: {posts_found}")

        # Stop only if we've scrolled at least `min_scrolls`
        if new_height == last_height:
            stall_counter += 1
            if i + 1 >= min_scrolls:
                print("No new posts detected - stopping scroll")
                break
        else:
            stall_counter = 0  # reset if progress is made

        last_height = new_height

# --------------------------
# 2. AUTHOR EXTRACTION  
# --------------------------
def get_author(post):
    """hunt down author name through 4 different methods"""
    # method 1: standard strong tag (works 60% of the time)
    author_elem = post.find("span", {"dir": "auto"})
    if author_elem:
        author_strong = author_elem.find("strong")
        if author_strong:
            return author_strong.get_text(strip=True)

    # method 2: aria-label sneak attack
    alt_author = post.find("a", {"aria-label": True})
    if alt_author and len(alt_author["aria-label"]) < 50:  # filter out long aria-labels
        return alt_author["aria-label"].strip()

    # method 3: role="link" backup
    role_author = post.find("span", {"role": "link"})
    if role_author:
        text = role_author.get_text(strip=True)
        if text and not text.isnumeric():  # filter out random numbers
            return text

    # method 4: desperate anchor tag grab
    fallback = post.find("a")
    if fallback:
        text = fallback.get_text(strip=True)
        if text and len(text) < 30 and not text.startswith(("http", "www")):  # basic sanity checks
            return text

    return "unknown"  # we tried our best 😔

# --------------------------  
# 3. TIMESTAMP CONVERSION
# --------------------------
def get_timestamp(post):
    """convert facebook's vague time hints into actual dates"""
    time_elem = post.find("a", href=lambda x: x and ("/posts/" in x or "/permalink/" in x))
    if not time_elem:
        return "unknown"
    
    timestamp = time_elem.get_text(strip=True)
    
    # handle "Just now" case
    if "Just now" in timestamp:
        return datetime.now().strftime("%Y-%m-%d %H:%M")
    
    # extract number and unit
    match = re.match(r"(\d+)\s*([smhdwy])", timestamp)
    if not match:
        return timestamp  # return as-is if we can't parse
    
    num, unit = int(match.group(1)), match.group(2)
    
    # calculate actual datetime
    now = datetime.now()
    if unit == 's':  # seconds
        return (now - timedelta(seconds=num)).strftime("%Y-%m-%d %H:%M")
    elif unit == 'm':  # minutes
        return (now - timedelta(minutes=num)).strftime("%Y-%m-%d %H:%M") 
    elif unit == 'h':  # hours
        return (now - timedelta(hours=num)).strftime("%Y-%m-%d %H:%M")
    elif unit == 'd':  # days
        return (now - timedelta(days=num)).strftime("%Y-%m-%d")
    elif unit == 'w':  # weeks
        return (now - timedelta(weeks=num)).strftime("%Y-%m-%d")
    elif unit == 'y':  # years
        return (now - timedelta(days=num*365)).strftime("%Y-%m-%d")
    
    return timestamp  # fallback

# --------------------------
# 4. REACTION SCRAPING
# --------------------------  
def get_reactions(post):
    """count reactions using every trick we know"""
    # method 1: aria-label (most reliable)
    reactions_elem = post.find("span", {"aria-label": lambda x: x and "reaction" in x.lower()})
    if reactions_elem:
        try:
            # extract first number found in aria-label
            numbers = re.findall(r'\d+', reactions_elem["aria-label"])
            if numbers:
                return int(numbers[0])
        except:
            pass
    
    # method 2: count reaction emoji images
    emoji_container = post.find("div", {"role": "button"})
    if emoji_container:
        return len(emoji_container.find_all("img", {"role": "presentation"}))
    
    # method 3: look for reaction text
    reaction_text = post.find(string=re.compile(r"Like|Love|Care|Haha|Wow|Sad|Angry", re.I))
    if reaction_text:
        return 1  # at least 1 reaction exists
    
    return 0  # probably no reactions

# --------------------------
# 5. MAIN EXTRACTION LOGIC
# --------------------------
def get_content(post):
    """extract post text with 'See more' expansion"""
    content_div = post.find("div", {"dir": "auto"})
    if not content_div:
        return ""
    
    content = content_div.get_text("\n", strip=True)
    
    # expand "See more" if present
    see_more = post.find("div", string=re.compile(r"See\s+more", re.I))
    if see_more:
        more_text = see_more.find_next("div")
        if more_text:
            content += "\n" + more_text.get_text("\n", strip=True)
    
    return content

def get_post_url(post):
    """extract post URL with fallback"""
    time_elem = post.find("a", href=lambda x: x and ("/posts/" in x or "/permalink/" in x))
    if time_elem:
        return urljoin("https://www.facebook.com", time_elem['href'])
    return "unknown"

def extract_posts(driver):
    """More reliable post detection with Facebook's current layout"""
    soup = BeautifulSoup(driver.page_source, "html.parser")
    posts_data = []
    
    # Try multiple selectors - Facebook changes these frequently
    post_selectors = [
        {'role': 'article'},  # Standard
        {'data-pagelet': re.compile('FeedUnit_')},  # Mobile/new layout
        {'class': re.compile('x1yztbdb')}  # Fallback class
    ]
    
    for selector in post_selectors:
        posts = soup.find_all("div", selector)
        if posts:
            break
    
    for post in posts:
        try:
            post_data = {
                "author": get_author(post),
                "timestamp": get_timestamp(post),
                "content": get_content(post),
                "post_url": get_post_url(post),
                "reactions": get_reactions(post)
            }
            posts_data.append(post_data)
        except Exception as e:
            continue
            
    return posts_data

# --------------------------
# 6. OUTPUT FUNCTIONS
# --------------------------
def save_posts_to_csv(posts, filename="fb_scraped_data.csv"):
    """save our hard-earned data to CSV"""
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["author", "timestamp", "content", "post_url", "reactions"])
        writer.writeheader()
        writer.writerows(posts)

# --------------------------
# 7. USER-FRIENDLY FILENAME HANDLING
# --------------------------
def get_output_filename():
    """get filename from user with smart defaults and validation"""
    while True:
        user_input = input("Enter output filename (or press Enter for 'fb_scraped_data.csv'): ").strip()
        
        # Use default if empty
        if not user_input:
            return "fb_scraped_data.csv"
        
        # Add .csv if missing
        if not user_input.lower().endswith('.csv'):
            user_input += '.csv'
        
        # Basic validation
        if len(user_input) > 100:
            print("That filename is too long. Try something shorter.")
            continue
            
        if not re.match(r'^[\w\-\. ]+$', user_input):
            print("Invalid characters in filename. Use only letters, numbers, spaces, hyphens, and periods.")
            continue
            
        return user_input

# --------------------------
# UPDATED MAIN EXECUTION FLOW
# --------------------------
def run_scraper(driver):
    try:
        print("🚀 Starting extended Facebook scraper (batch scroll mode)")
        total_scrolls = 600
        scrolls_per_batch = 10
        total_batches = total_scrolls // scrolls_per_batch
        wait = 2.5  # you can bump to 3 if needed

        all_posts = {}
        last_count = 0
        stale_batches = 0

        for batch in range(total_batches):
            print(f"Batch {batch+1}/{total_batches} | Scrolling {scrolls_per_batch} times...")
            scroll_to_load(driver, max_scrolls=scrolls_per_batch, wait=wait, verbose=False)
            
            print("🔍 Extracting posts...")
            new_posts = extract_posts(driver)
            
            new_count = 0
            for p in new_posts:
                if p["post_url"] not in all_posts:
                    all_posts[p["post_url"]] = p
                    new_count += 1

            total_unique = len(all_posts)
            print(f"New: {new_count} | Total unique: {total_unique}")

            # Early stop if no new posts for 3 consecutive batches
            if new_count == 0:
                stale_batches += 1
                print(f"⚠️ No new posts in this batch ({stale_batches}x)")
                if stale_batches >= 3:
                    print("No new content in 3 batches. Stopping early.")
                    break
            else:
                stale_batches = 0

        posts_list = list(all_posts.values())

        print(f"\n✅ Finished scraping: {len(posts_list)} unique posts")

        if posts_list:
            filename = get_output_filename()
            save_posts_to_csv(posts_list, filename)
            print(f"Saved {len(posts_list)} posts to '{filename}'")

            # Optional summary
            print("\nTop timestamps:")
            df = pd.DataFrame(posts_list)
            print(df['timestamp'].value_counts().head(10))

    except Exception as e:
        print(f"Error: {str(e)}")

### run scraper

In [ ]:
run_scraper(driver)

🚀 Starting Continuous Scroll Scraper v2.4
🔍 Loading posts (this will continue until no new posts appear)...
🔄 Scroll 48/300 | Position: 38184px | Posts: 12
💤 No new posts detected - stopping at 38184px

✨ Found 14 posts total

✅ Found 11 unique posts


In [58]:
df = pd.read_csv('otsego_good_maybe.csv')
df

,author,timestamp,content,post_url,reactions
0,unknown,2025-01-26,Before the lynx was a tree farm what was it? I...,https://www.facebook.com/groups/19969065872295...,0
1,10w,2025-01-26,I need help getting the map,https://www.facebook.com/groups/19969065872295...,0
2,unknown,2025-01-26,Did they buy or were they granted access by th...,https://www.facebook.com/groups/19969065872295...,0
3,unknown,2025-02-02,Thank you\nChristina DeGrush\nI love you,https://www.facebook.com/groups/19969065872295...,0
4,unknown,2025-02-02,Thank you!,https://www.facebook.com/groups/19969065872295...,0
...,...,...,...,...,...
252,unknown,2018-04-08,Don't Forget about the old rock ten paper mill...,https://www.facebook.com/groups/19969065872295...,0
253,unknown,2018-04-08,We all feared there would be a lot of conspira...,https://www.facebook.com/groups/19969065872295...,0
254,unknown,2018-04-08,Everything seems so obvious. Let’s get this done.,https://www.facebook.com/groups/19969065872295...,0
255,unknown,2018-04-08,It's Thursday and I won't get down to Micheal'...,https://www.facebook.com/groups/19969065872295...,0
